In [1]:
import numpy as np 
import pandas as pd 
import os
import gc
import psutil

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

from keras.models import Model
from keras.layers import Input, Dropout, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, BatchNormalization, Conv1D, MaxPooling1D, Flatten
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
from keras.models import Model

from keras import optimizers

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler

import warnings
warnings.filterwarnings('ignore')

from keras import backend as K
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [2]:
def RNN_model():

    ##Inputs
    #text
    title = Input(shape=[16], name="title")
    description = Input(shape=[128], name="description")
    #category        
    parent_category_name=Input(shape=[1], name="parent_category_name")
    category_name=Input(shape=[1], name="category_name")
    param_1 = Input(shape=[1], name="param_1")
    param123 = Input(shape=[1], name="param123")
    image_top_1 = Input(shape=[1], name="image_top_1")
    cluster_5 = Input(shape=[1], name="cluster_5")
    cluster_10 = Input(shape=[1], name="cluster_10")
    cluster_20 = Input(shape=[1], name="cluster_20")
    user_type = Input(shape=[1], name="user_type")
    activation_date = Input(shape=[1], name="activation_date")
    #numerical
    #'para_num','item_seq_number',   
    #'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
    #'dis_days', 'edit_time', 'item_num',
    #'count',  'cus', 'ship', 'r_cus', 'r_ship', 'all_cus', 
    #'avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
    #'avg_pixel_width', 'width', 'height', 'size', 
    #'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2', 
    #'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2', 
    #'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2',
    #'description_num_words','description_num_unique_words','description_words_vs_unique',
    #'title_num_words','title_num_unique_words','title_words_vs_unique',
    #'price'
    numerical= Input(shape=[47], name="numerical")
    
    
    #Embeddings layers
    vocab_size=200001
    EMBEDDING_DIM1=300
    emb_title = Embedding(vocab_size, EMBEDDING_DIM1, weights = [embedding_matrix1], 
                                          trainable = False)(title)
    emb_description = Embedding(vocab_size, EMBEDDING_DIM1, weights = [embedding_matrix1], 
                                          trainable = False)(description)
    emb_category_name = Embedding(48, 5)(category_name)
    emb_parent_category_name = Embedding(10, 5)(parent_category_name)
    emb_param_1 = Embedding(373, 3)(param_1)
    emb_param123 = Embedding(2402, 3)(param123)
    emb_image_top_1 = Embedding(3065, 3)(image_top_1)    
    emb_cluster_5 = Embedding(68, 3)(cluster_5)
    emb_cluster_10 = Embedding(47, 3)(cluster_10)
    emb_cluster_20 = Embedding(24, 3)(cluster_20)
    emb_user_type = Embedding(4, 5)(user_type)
    emb_activation_date = Embedding(8, 3)(activation_date)
       

    rnn_layer1 = GRU(8) (emb_title)
    rnn_layer2 = GRU(8) (emb_description)
    
    
    
    #main layer
    main_l = concatenate([rnn_layer1,rnn_layer2, 
                          Flatten() (emb_category_name), 
                          Flatten() (emb_parent_category_name), 
                          Flatten() (emb_param_1), 
                          Flatten() (emb_param123), 
                          Flatten() (emb_image_top_1), 
                          Flatten() (emb_cluster_5), 
                          Flatten() (emb_cluster_10), 
                          Flatten() (emb_cluster_20), 
                          Flatten() (emb_user_type), 
                          Flatten() (emb_activation_date), 
                          numerical])
    out_rnn=concatenate([rnn_layer1,rnn_layer2])
    
    main_l = Dropout(0.30)(Dense(64,activation='relu') (main_l))
    main_l = Dropout(0.30)(Dense(32,activation='relu') (main_l))
    main_l = Dropout(0.30)(Dense(16,activation='relu') (main_l))
    main_l = Dense(4,activation='relu') (main_l)
    
    #output
    output = Dense(1,activation="sigmoid") (main_l)
    
    #model
    
    model = Model(inputs=[title, description, parent_category_name, category_name, param_1, param123, image_top_1, 
                         cluster_5, cluster_10, cluster_20, user_type, activation_date, numerical],
                  outputs=[output, out_rnn])
    model.compile(optimizer = 'adam', loss= root_mean_squared_error, metrics = [root_mean_squared_error],loss_weights=[1.0,0.0])
    return model

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_true - y_pred))) 

In [3]:
print(os.listdir('E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'))

['embedding_matrix1.npy', 'testfinal.csv', 'testready.csv', 'testtemp.csv', 'trainfinal.csv', 'trainready.csv', 'traintemp.csv']


In [4]:

def get_keras_data(data1,data2,data3):
    title=[]
    for i in list(data1.title):
        title.append(list(eval(i)))
    description=[]
    for i in list(data1.description):
        description.append(list(eval(i)))
    
    title=pad_sequences(title, maxlen=16)
    description=pad_sequences(description, maxlen=128)
    parent_category_name=np.array(data2.parent_category_name)
    category_name=np.array(data2.category_name)
    param_1=np.array(data2.param_1)
    param123=np.array(data2.param123)
    image_top_1=np.array(data2.image_top_1)
    cluster_5=np.array(data2.cluster_5)
    cluster_10=np.array(data2.cluster_10)
    cluster_20=np.array(data2.cluster_20)
    user_type=np.array(data2.user_type)
    activation_date=np.array(data2.activation_date)
    numerical=np.array(data3)
    
    
    output=np.array(data2.deal_probability)
    out_rnn=np.zeros((len(data1), 16))
    
    print("Data ready for Vectorization")
    
    X=[title, description, parent_category_name, category_name, param_1, param123, image_top_1,cluster_5, cluster_10, cluster_20, user_type, activation_date, numerical]
    Y=[output, out_rnn]
    return (X,Y)

In [5]:
columns1=['title', 'description']
columns2=['parent_category_name','category_name', 'user_type','activation_date',
         'param_1', 'param123','image_top_1','cluster_5', 'cluster_10','cluster_20','deal_probability']
columns3=['para_num','item_seq_number',   
         'deal_prob_avg', 'deal_prob_avg_par', 'count_avg', 'count_avg_par',
         'dis_days', 'edit_time', 'item_num',
         'count',  'cus', 'ship', 'r_cus', 'r_ship', 'all_cus', 
         'avg_r','avg_g', 'avg_b', 'blurrness', 'dullness', 'whiteness',
         'avg_pixel_width', 'width', 'height', 'size', 
         'resnet_1', 'resnet_3','resnet_5', 'resnet_ratio1', 'resnet_ratio2', 
         'incept_1', 'incept_3','incept_5', 'incept_ratio1', 'incept_ratio2', 
         'xcept_1', 'xcept_3','xcept_5', 'xcept_ratio1', 'xcept_ratio2',
         'description_num_words','description_num_unique_words','description_words_vs_unique',
         'title_num_words','title_num_unique_words','title_words_vs_unique',
         'price']
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
data1=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns1)
data2=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns2)
data3=pd.read_csv(path+'trainfinal.csv',header=0,skipinitialspace=True, usecols=columns3)

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

skf = KFold(n_splits = 5, shuffle=True,random_state=123456789)
count=0
#seed=np.random.randint(0,10)
seed=1
for train_idx, test_idx in skf.split(data3, data2.deal_probability):
    if count==seed:
        train_id=train_idx
        test_id=test_idx
    count=count+1
        
gc.collect()

data1_train, data1_test = data1.ix[train_id], data1.ix[test_id]
data2_train, data2_test = data2.ix[train_id], data2.ix[test_id]
data3_train, data3_test = data3.ix[train_id], data3.ix[test_id]
del data1,data2,data3
gc.collect()

X_train, Y_train = get_keras_data(data1_train,data2_train,data3_train)
X_test, Y_test = get_keras_data(data1_test,data2_test,data3_test)


del data1_train,data2_train,data3_train,data1_test,data2_test,data3_test

gc.collect()


Data ready for Vectorization
Data ready for Vectorization


21

In [6]:
path='E:\\kaggle\\Avito Demand Prediction Challenge\\features\\dataset\\'
embedding_matrix1=np.load(path+'embedding_matrix1.npy')
gc.collect()


3

In [7]:
exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1

epochs = 50
batch_size = 4096
steps = ((len(Y_train[0]))/batch_size)*epochs
lr_init, lr_fin = 0.005, 0.00000005
lr_decay = exp_decay(lr_init, lr_fin, steps)
modelRNN = RNN_model()
K.set_value(modelRNN.optimizer.lr, lr_init)
K.set_value(modelRNN.optimizer.decay, lr_decay)

In [ ]:
hist = modelRNN.fit(X_train,Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test),verbose=1)





Train on 1202739 samples, validate on 300685 samples
Epoch 1/50
1202739/1202739 [==============================] - 98s 82us/step - loss: 0.2440 - dense_5_loss: 0.2440 - concatenate_2_loss: 0.2909 - dense_5_root_mean_squared_error: 0.2440 - concatenate_2_root_mean_squared_error: 0.2909 - val_loss: 0.2294 - val_dense_5_loss: 0.2294 - val_concatenate_2_loss: 0.2918 - val_dense_5_root_mean_squared_error: 0.2294 - val_concatenate_2_root_mean_squared_error: 0.2918
Epoch 2/50
1202739/1202739 [==============================] - 97s 80us/step - loss: 0.2298 - dense_5_loss: 0.2298 - concatenate_2_loss: 0.2872 - dense_5_root_mean_squared_error: 0.2298 - concatenate_2_root_mean_squared_error: 0.2872 - val_loss: 0.2306 - val_dense_5_loss: 0.2306 - val_concatenate_2_loss: 0.2900 - val_dense_5_root_mean_squared_error: 0.2306 - val_concatenate_2_root_mean_squared_error: 0.2900
Epoch 3/50
1202739/1202739 [==============================] - 97s 80us/step - loss: 0.2280 - dense_5_loss: 0.2280 - concatenate

Epoch 21/50
1202739/1202739 [==============================] - 96s 80us/step - loss: 0.2197 - dense_5_loss: 0.2197 - concatenate_2_loss: 0.3076 - dense_5_root_mean_squared_error: 0.2197 - concatenate_2_root_mean_squared_error: 0.3076 - val_loss: 0.2259 - val_dense_5_loss: 0.2259 - val_concatenate_2_loss: 0.3074 - val_dense_5_root_mean_squared_error: 0.2259 - val_concatenate_2_root_mean_squared_error: 0.3074
Epoch 22/50
1202739/1202739 [==============================] - 96s 80us/step - loss: 0.2196 - dense_5_loss: 0.2196 - concatenate_2_loss: 0.3083 - dense_5_root_mean_squared_error: 0.2196 - concatenate_2_root_mean_squared_error: 0.3083 - val_loss: 0.2297 - val_dense_5_loss: 0.2297 - val_concatenate_2_loss: 0.3087 - val_dense_5_root_mean_squared_error: 0.2297 - val_concatenate_2_root_mean_squared_error: 0.3087
Epoch 23/50
1202739/1202739 [==============================] - 96s 80us/step - loss: 0.2192 - dense_5_loss: 0.2192 - concatenate_2_loss: 0.3090 - dense_5_root_mean_squared_error:

Epoch 41/50
1202739/1202739 [==============================] - 96s 79us/step - loss: 0.2167 - dense_5_loss: 0.2167 - concatenate_2_loss: 0.3319 - dense_5_root_mean_squared_error: 0.2167 - concatenate_2_root_mean_squared_error: 0.3319 - val_loss: 0.2262 - val_dense_5_loss: 0.2262 - val_concatenate_2_loss: 0.3284 - val_dense_5_root_mean_squared_error: 0.2262 - val_concatenate_2_root_mean_squared_error: 0.3284
Epoch 42/50
1085440/1202739 [==========================>...] - ETA: 8s - loss: 0.2169 - dense_5_loss: 0.2169 - concatenate_2_loss: 0.3321 - dense_5_root_mean_squared_error: 0.2169 - concatenate_2_root_mean_squared_error: 0.3321

In [9]:
val_preds,rnn = modelRNN.predict(X_test)

In [26]:
def rmse(y, y_pred):
    y=y.flatten()
    y_pred=y_pred.flatten()

    Rsum = np.sum((y - y_pred)**2)
    n = len(y)
    RMSE = np.sqrt(Rsum/n)
    return RMSE 

rmse(Y_test[0], val_preds)

In [18]:
model.save(filepath)
keras.models.load_model(filepath)

numpy.ndarray

In [21]:
(Y_test[0])

array([0.80323, 0.     , 0.10342, ..., 0.     , 0.     , 0.7376 ])

In [23]:
val_preds.flatten()


array([0.18857522, 0.06045391, 0.16316088, ..., 0.1368806 , 0.123833  ,
       0.39630845], dtype=float32)

0.2237126912997403